In [733]:
from IPython.display import display, Math, Latex

In [734]:
import pandas as pd
df = pd.read_csv('Acoustic Features.csv')

In [735]:
 df

,Class,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,...,0.426,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970
1,relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,...,0.002,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967
2,relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,...,0.184,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963
3,relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,...,0.038,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968
4,relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,...,0.004,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,angry,0.174,0.578,5.265,161.144,2.985,0.065,0.718,0.046,0.265,...,0.374,0.838,0.413,0.125,0.323,0.140,0.120,2.691,0.867,0.969
396,angry,0.176,0.498,5.042,179.787,2.790,-0.148,0.342,-0.011,0.029,...,0.020,1.000,0.640,0.010,0.233,0.127,0.145,3.364,0.879,0.967
397,angry,0.187,0.557,4.724,134.032,1.672,0.566,0.880,-0.113,0.244,...,0.052,0.184,0.287,0.101,0.269,0.157,0.111,3.364,0.819,0.962
398,angry,0.140,0.573,4.470,113.600,1.997,-0.210,0.694,0.089,0.206,...,0.137,1.000,0.304,0.140,0.277,0.144,0.061,3.364,0.811,0.969


In [736]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 51 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Class                                           400 non-null    object 
 1   _RMSenergy_Mean                                 400 non-null    float64
 2   _Lowenergy_Mean                                 400 non-null    float64
 3   _Fluctuation_Mean                               400 non-null    float64
 4   _Tempo_Mean                                     400 non-null    float64
 5   _MFCC_Mean_1                                    400 non-null    float64
 6   _MFCC_Mean_2                                    400 non-null    float64
 7   _MFCC_Mean_3                                    400 non-null    float64
 8   _MFCC_Mean_4                                    400 non-null    float64
 9   _MFCC_Mean_5                               

## **Pre-processing**

**Normalisasi data dengan Z-score**
$$ X(stand) = \frac{x - mean (x)}{std_\text{dev}} $$
dimana $${mean = 0}, {std_\text{dev}} = 1 $$

In [737]:
def normalize(data, i):
    # Initialize a list to store the means of each column
    means = [0] * i
    # Initialize a list to store the standard deviations of each column
    stds = [1] * i
    # Loop through the columns
    for j in range(i):
        # Compute the mean of the column
        mean = data.iloc[:, j].mean()
        # Compute the standard deviation of the column
        std = data.iloc[:, j].std()
        # Store the mean and standard deviation
        means[j] = mean
        stds[j] = std
    # Loop through the rows
    for index, row in data.iterrows():
        # Loop through the columns
        for j in range(i):
            # Normalize the value by subtracting the mean and dividing by the standard deviation
            row[j] = (row[j] - means[j]) / stds[j]
    # Return the normalized data
    return data

Setelah data dinormalisasi, selanjutnya ialah PCA

- menghitung covariance matrix
\begin{align}
        Cov (\sum) = \begin{pmatrix}
        cov (x_1,x_1) & cov(x_1,x_2) & ... & cov(x_1,x_m) \\
        | & | & | & | \\
        cov (x_m,x_1) & cov(x_m,x_2) & ... & cov(x_m,x_m)
        \end{pmatrix}
    \end{align}
  di mana \(X\) adalah matriks data yang memiliki dimensi \(n x m\), dengan \(n\) adalah jumlah sampel dan \(m\) adalah jumlah fitur.
- menghitung eigenvalue dan eigenvector
$$ AX = λX $$

In [738]:
# Import numpy and pandas libraries
import numpy as np
import pandas as pd

# Define a function to do a PCA
def do_pca(data, n_components):
  # data: a numpy array of shape (n_samples, n_features)
  # n_components: an integer specifying the number of principal components to keep
  # returns: a numpy array of shape (n_samples, n_components) containing the projected data

  # Calculate the covariance matrix of the data
  cov_matrix = np.cov(data.T)

  # Calculate the eigenvalues and eigenvectors of the covariance matrix
  eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

  # Sort the eigenvalues and eigenvectors in descending order
  idx = np.argsort(eigenvalues)[::-1]
  eigenvalues = eigenvalues[idx]
  eigenvectors = eigenvectors[:, idx]

  # Select the first n_components eigenvectors as the principal components
  pc = eigenvectors[:, :n_components]

  # Project the data onto the principal components
  transformed_data = np.dot(data, pc)

  # Return the transformed data
  return transformed_data


In [739]:
# check duplicate value
df.duplicated().sum()

12

In [740]:
# drop duplicate
df = df.drop_duplicates(keep='first')

In [741]:
# Split feature and target
X = df.drop(columns=['Class'])
y = df['Class']

In [742]:
X = normalize(X,50)
X

,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,_MFCC_Mean_6,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,-1.272863,0.724549,0.910332,0.184485,1.940591,0.533531,1.342709,0.176184,0.194331,0.383737,...,0.207447,1.143513,-1.063516,-0.825846,-0.221041,1.447311,0.166633,-0.794990,-2.447998,0.849226
1,-0.141269,-2.270546,-0.198455,0.542841,2.017159,0.817391,0.998348,1.320472,1.910184,3.700735,...,-1.057144,1.143513,-1.088927,1.695816,-0.779237,0.384200,-0.791482,1.701411,-0.935784,0.074540
2,-1.365871,1.670369,1.561337,1.891824,0.406721,1.535389,0.042915,1.076549,0.532379,0.490894,...,-0.514324,0.435849,-1.038105,1.741509,1.525571,2.255275,1.278046,-0.101486,-1.074519,-0.958374
3,0.013743,0.961004,1.499939,-1.697403,0.489565,2.739475,0.414284,1.155465,-0.881279,0.505506,...,-0.949773,1.143513,-0.577529,1.047552,1.687627,1.532360,0.396581,-1.524938,-0.366969,0.332769
4,-1.055845,0.724549,1.196106,-1.024616,0.961525,0.283066,1.096254,0.240752,1.285306,1.197156,...,-1.051179,-0.516990,2.087458,-1.111424,0.301142,1.447311,0.846895,-1.102619,-1.324242,-2.507745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.618294,0.468390,-0.837272,1.098253,0.670316,-0.019347,0.772150,0.061397,0.419696,0.315546,...,0.052356,0.692168,0.222920,-0.757307,-0.094997,-1.125418,1.143910,0.980038,1.353348,0.590997
396,0.649296,-1.107976,-0.937947,1.645996,0.425549,-0.414524,-0.497260,-0.143068,-0.789084,-0.001054,...,-1.003458,1.143513,0.943959,-1.085722,-1.715565,-1.401827,1.383439,1.701411,1.519830,0.074540
397,0.819810,0.054594,-1.081512,0.301685,-0.977778,0.910155,1.319076,-0.508954,0.312135,0.851331,...,-0.908018,-1.129927,-0.177304,-0.825846,-1.067338,-0.763960,1.057680,1.701411,0.687419,-1.216602
398,0.091250,0.369867,-1.196183,-0.298621,-0.569834,-0.529552,0.691124,0.215643,0.117501,0.110973,...,-0.654503,1.143513,-0.123306,-0.714470,-0.923287,-1.040369,0.578622,1.701411,0.576431,0.590997


### **One-hot Encoding for y**

In [743]:
y = pd.get_dummies(y, columns = ['Class'])
y

,angry,happy,relax,sad
0,False,False,True,False
1,False,False,True,False
2,False,False,True,False
3,False,False,True,False
4,False,False,True,False
...,...,...,...,...
395,True,False,False,False
396,True,False,False,False
397,True,False,False,False
398,True,False,False,False


# Reduction for highly corellated data

Setelah melakukan perhitungan matriks korelasi, heatmap digambarkan dan dilihat secara total fitur apa yang memiliki korelasi tinggi. Kode dibawah hanya menampilkan korelasi heatmap untuk data yang memiliki korelasi tinggi diatas 0.8

In [744]:
import seaborn as sns
import matplotlib.pyplot as plt

data = X[['_Rolloff_Mean','_Eventdensity_Mean','_Pulseclarity_Mean','_Brightness_Mean','_Spectralcentroid_Mean','_Spectralspread_Mean','_Spectralskewness_Mean','_Spectralkurtosis_Mean','_Spectralflatness_Mean','_EntropyofSpectrum_Mean','_MFCC_Mean_1']]
matrix = data.corr(method='pearson')
plt.figure(figsize=(100,100))
sns.heatmap(matrix, annot=True,linewidths=.5)
plt.show()

In [745]:
trial = X[['_Rolloff_Mean','_Brightness_Mean','_Spectralcentroid_Mean','_Spectralspread_Mean','_Spectralskewness_Mean','_Spectralkurtosis_Mean','_EntropyofSpectrum_Mean','_MFCC_Mean_1']]
result = do_pca(trial,1)

In [746]:
X_with_PCA = X.drop(['_Rolloff_Mean','_Brightness_Mean','_Spectralcentroid_Mean','_Spectralspread_Mean','_Spectralskewness_Mean','_Spectralkurtosis_Mean','_EntropyofSpectrum_Mean','_MFCC_Mean_1'],axis=1)
X_with_PCA["PCA_Val"] = result

In [747]:
X

,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_1,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,_MFCC_Mean_6,...,_Chromagram_Mean_9,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy
0,-1.272863,0.724549,0.910332,0.184485,1.940591,0.533531,1.342709,0.176184,0.194331,0.383737,...,0.207447,1.143513,-1.063516,-0.825846,-0.221041,1.447311,0.166633,-0.794990,-2.447998,0.849226
1,-0.141269,-2.270546,-0.198455,0.542841,2.017159,0.817391,0.998348,1.320472,1.910184,3.700735,...,-1.057144,1.143513,-1.088927,1.695816,-0.779237,0.384200,-0.791482,1.701411,-0.935784,0.074540
2,-1.365871,1.670369,1.561337,1.891824,0.406721,1.535389,0.042915,1.076549,0.532379,0.490894,...,-0.514324,0.435849,-1.038105,1.741509,1.525571,2.255275,1.278046,-0.101486,-1.074519,-0.958374
3,0.013743,0.961004,1.499939,-1.697403,0.489565,2.739475,0.414284,1.155465,-0.881279,0.505506,...,-0.949773,1.143513,-0.577529,1.047552,1.687627,1.532360,0.396581,-1.524938,-0.366969,0.332769
4,-1.055845,0.724549,1.196106,-1.024616,0.961525,0.283066,1.096254,0.240752,1.285306,1.197156,...,-1.051179,-0.516990,2.087458,-1.111424,0.301142,1.447311,0.846895,-1.102619,-1.324242,-2.507745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.618294,0.468390,-0.837272,1.098253,0.670316,-0.019347,0.772150,0.061397,0.419696,0.315546,...,0.052356,0.692168,0.222920,-0.757307,-0.094997,-1.125418,1.143910,0.980038,1.353348,0.590997
396,0.649296,-1.107976,-0.937947,1.645996,0.425549,-0.414524,-0.497260,-0.143068,-0.789084,-0.001054,...,-1.003458,1.143513,0.943959,-1.085722,-1.715565,-1.401827,1.383439,1.701411,1.519830,0.074540
397,0.819810,0.054594,-1.081512,0.301685,-0.977778,0.910155,1.319076,-0.508954,0.312135,0.851331,...,-0.908018,-1.129927,-0.177304,-0.825846,-1.067338,-0.763960,1.057680,1.701411,0.687419,-1.216602
398,0.091250,0.369867,-1.196183,-0.298621,-0.569834,-0.529552,0.691124,0.215643,0.117501,0.110973,...,-0.654503,1.143513,-0.123306,-0.714470,-0.923287,-1.040369,0.578622,1.701411,0.576431,0.590997


In [748]:
X_with_PCA

,_RMSenergy_Mean,_Lowenergy_Mean,_Fluctuation_Mean,_Tempo_Mean,_MFCC_Mean_2,_MFCC_Mean_3,_MFCC_Mean_4,_MFCC_Mean_5,_MFCC_Mean_6,_MFCC_Mean_7,...,_Chromagram_Mean_10,_Chromagram_Mean_11,_Chromagram_Mean_12,_HarmonicChangeDetectionFunction_Mean,_HarmonicChangeDetectionFunction_Std,_HarmonicChangeDetectionFunction_Slope,_HarmonicChangeDetectionFunction_PeriodFreq,_HarmonicChangeDetectionFunction_PeriodAmp,_HarmonicChangeDetectionFunction_PeriodEntropy,PCA_Val
0,-1.272863,0.724549,0.910332,0.184485,0.533531,1.342709,0.176184,0.194331,0.383737,-1.159007,...,1.143513,-1.063516,-0.825846,-0.221041,1.447311,0.166633,-0.794990,-2.447998,0.849226,-4.910212
1,-0.141269,-2.270546,-0.198455,0.542841,0.817391,0.998348,1.320472,1.910184,3.700735,-2.543376,...,1.143513,-1.088927,1.695816,-0.779237,0.384200,-0.791482,1.701411,-0.935784,0.074540,-6.073207
2,-1.365871,1.670369,1.561337,1.891824,1.535389,0.042915,1.076549,0.532379,0.490894,-1.680892,...,0.435849,-1.038105,1.741509,1.525571,2.255275,1.278046,-0.101486,-1.074519,-0.958374,3.065147
3,0.013743,0.961004,1.499939,-1.697403,2.739475,0.414284,1.155465,-0.881279,0.505506,-0.071288,...,1.143513,-0.577529,1.047552,1.687627,1.532360,0.396581,-1.524938,-0.366969,0.332769,-0.242623
4,-1.055845,0.724549,1.196106,-1.024616,0.283066,1.096254,0.240752,1.285306,1.197156,0.241843,...,-0.516990,2.087458,-1.111424,0.301142,1.447311,0.846895,-1.102619,-1.324242,-2.507745,-2.510837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.618294,0.468390,-0.837272,1.098253,-0.019347,0.772150,0.061397,0.419696,0.315546,0.758234,...,0.692168,0.222920,-0.757307,-0.094997,-1.125418,1.143910,0.980038,1.353348,0.590997,-0.765846
396,0.649296,-1.107976,-0.937947,1.645996,-0.414524,-0.497260,-0.143068,-0.789084,-0.001054,0.027595,...,1.143513,0.943959,-1.085722,-1.715565,-1.401827,1.383439,1.701411,1.519830,0.074540,2.820960
397,0.819810,0.054594,-1.081512,0.301685,0.910155,1.319076,-0.508954,0.312135,0.851331,1.197716,...,-1.129927,-0.177304,-0.825846,-1.067338,-0.763960,1.057680,1.701411,0.687419,-1.216602,1.796603
398,0.091250,0.369867,-1.196183,-0.298621,-0.529552,0.691124,0.215643,0.117501,0.110973,0.620896,...,1.143513,-0.123306,-0.714470,-0.923287,-1.040369,0.578622,1.701411,0.576431,0.590997,0.401799


### **Training**

**Fungsi Aktifasi:**

- Sigmoid:
$$\sigma(x) = \frac{1}{1 + e^{-x}} $$
$$\sigma'(x) = \sigma(x) \cdot (1 - \sigma(x))$$
- Softmax:
$${softmax}(x_i) = \frac{e^{x_i - \max(x)}}{\sum{e^{x - \max(x)}}} $$
$$ softmax'(x) = $$
- ReLU
$${ReLU}(x) = \max(0, x) $$
$${ReLU}'(x) = \begin{cases} 1 & \text{jika } x > 0 \\ 0 & \text{jika } x \leq 0 \end{cases} $$


**Loss Function:**
- Categorical Cross-Entropy:
$${CCE}(y_{\text{true}}, y_{\text{pred}}) = -\frac{1}{N} \sum{y_{\text{true}} \cdot \log(y_{\text{pred}})} $$
- Turunan Categorical Cross-Entropy:
   $${CCE}'(y_{\text{true}}, y_{\text{pred}}) = -\frac{y_{\text{true}}}{y_{\text{pred}}}$$


In [749]:
# Import numpy for numerical operations
import numpy as np

# Define the activation functions and their derivatives
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def softmax_derivative(x):
    s = softmax(x)
    return s * (1 - s)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

# Define the loss functions and their derivatives
def categorical_crossentropy(y_true, y_pred):
    # Clip the predictions to avoid log(0) error
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Compute the cross entropy
    return -np.mean(y_true * np.log(y_pred))

def categorical_crossentropy_derivative(y_true, y_pred):
    # Clip the predictions to avoid division by zero error
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    # Compute the derivative of cross entropy
    return - (y_true / y_pred)


# Define the class for a layer of the neural network
class Layer:
    # Initialize the layer with the number of inputs, outputs, activation function, and weight initialization method
    def __init__(self, n_inputs, n_outputs, activation, weight_init):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.activation = activation
        self.weight_init = weight_init
        # Initialize the weights and biases according to the activation function and weight initialization method
        if activation == "sigmoid" or activation == "softmax" or activation == "tanh":
            # Use Xavier initialization
            self.weights = np.random.randn(n_inputs, n_outputs) * np.sqrt(1 / n_inputs)
            self.biases = np.zeros((1, n_outputs))
        elif activation == "relu":
            # Use He initialization
            self.weights = np.random.randn(n_inputs, n_outputs) * np.sqrt(2 / n_inputs)
            self.biases = np.zeros((1, n_outputs))
        else:
            # Use random initialization
            self.weights = np.random.randn(n_inputs, n_outputs)
            self.biases = np.zeros((1, n_outputs))
        # Initialize the gradients and the cache for the forward and backward pass
        self.dweights = np.zeros_like(self.weights)
        print('inisiasi dweights',np.shape(self.dweights))
        self.dbiases = np.zeros_like(self.biases)
        self.z = None
        self.a = None

    # Forward pass the inputs through the layer and apply the activation function
    def forward(self, inputs):
        #save input data
        self.input = inputs
        # Compute the linear combination of inputs, weights, and biases
        self.z = np.dot(inputs, self.weights) + self.biases
        # Apply the activation function
        if self.activation == "sigmoid":
            self.a = sigmoid(self.z)
        elif self.activation == "softmax":
            self.a = softmax(self.z)
        elif self.activation == "relu":
            self.a = relu(self.z)
        else:
            self.a = self.z
        # Return the outputs
        return self.a

    # Backward pass the error through the layer and compute the gradients
    def backward(self, error):#,inputs):
        # Apply the derivative of the activation function to the error
        if self.activation == "sigmoid":
            error = error * sigmoid_derivative(self.z)
        elif self.activation == "softmax":
            error = error * softmax_derivative(self.z)
        elif self.activation == "relu":
            error = error * relu_derivative(self.z)
        # Compute the gradients of weights and biases
        print('error size', np.shape(error))
        print('a size', np.shape(self.a))
        print('inputs.T <dot> error')
        self.dweights = np.dot(self.inputs.T,error)
        print('dw size', np.shape(self.dweights))
        self.dbiases = np.sum(error, axis=0, keepdims=True)
        # Compute the error for the previous layer
        error = np.dot(error, self.weights.T)
        print('previous layer err size', np.shape(error))
        # Return the error
        return error

    # Update the weights and biases with the gradients and the learning rate
    def update(self, learning_rate):
        print(' dweights',np.shape(self.dweights))
        print(' weight',np.shape(self.weights))
        self.weights = self.weights - learning_rate * self.dweights
        self.biases = self.biases - learning_rate * self.dbiases

# Define the class for the neural network
class NeuralNetwork:
    # Initialize the neural network with the number of features, the number of outputs, the loss function, and the optimizer
    def __init__(self, n_features, n_outputs, loss, optimizer):
        self.n_features = n_features
        self.n_outputs = n_outputs
        self.loss = loss
        self.optimizer = optimizer
        # Initialize the list of layers
        self.layers = []

    # Add a layer to the neural network with the number of outputs, the activation function, and the weight initialization method
    def add_layer(self, n_outputs, activation, weight_init):
        # Check if the neural network is empty
        if len(self.layers) == 0:
            # Use the number of features as the number of inputs for the first layer
            n_inputs = self.n_features
        else:
            # Use the number of outputs of the previous layer as the number of inputs for the next layer
            n_inputs = self.layers[-1].n_outputs
        # Create a new layer with the given parameters
        layer = Layer(n_inputs, n_outputs, activation, weight_init)
        # Append the layer to the list of layers
        self.layers.append(layer)

    # Train the neural network with the inputs, the outputs, the number of epochs, and the learning rate
    def train(self, x_train, y_train, epochs, learning_rate):
        # Loop over the number of epochs
        for epoch in range(epochs):
            # Forward pass the inputs through the neural network and get the predictions
            y_pred = self.forward(x_train)
            # Compute the loss
            if self.loss == "categorical_crossentropy":
                loss = categorical_crossentropy(y_train, y_pred)
            elif self.loss == "mse":
                loss = mse(y_train, y_pred)
            else:
                loss = None
            # Print the epoch and the loss
            print(f"Epoch {epoch + 1}, Loss: {loss}")
            # Backward pass the error through the neural network and compute the gradients
            self.backward(x_train, y_train)
            # Update the weights and biases with the gradients and the learning rate
            self.update(learning_rate)

    # Forward pass the inputs through the neural network and get the predictions
    def forward(self, inputs):
        # Loop over the layers
        for layer in self.layers:
            # Forward pass the inputs through the layer and get the outputs
            outputs = layer.forward(inputs)
            # Use the outputs as the inputs for the next layer
            inputs = outputs
        # Return the final outputs as the predictions
        return outputs

    # Backward pass the error through the neural network and compute the gradients
    def backward(self, inputs, outputs):
        # Compute the error
        if self.loss == "categorical_crossentropy":
            error = categorical_crossentropy_derivative(outputs, self.forward(inputs))
        elif self.loss == "mse":
            error = mse_derivative(outputs, self.forward(inputs))
        else:
            error = None

        # Loop over the layers in reverse order
        for i, layer in enumerate(reversed(self.layers)):
            print('backprop ', i)
            print('reverse init err',np.shape(error))
            # Backward pass the error through the layer and get the error for the previous layer
            error = layer.backward(error)#,inputs)
            print('reverse err', np.shape(error))

            # Use the activated outputs of the current layer as the inputs for the previous layer
            inputs = layer.a
            print('reverse input =', np.shape(inputs))
            print('resulting dweights', np.shape(layer.dweights))


    # Update the weights and biases with the gradients and the learning rate
    def update(self, learning_rate):
        # Loop over the layers
        for layer in (self.layers):
            # Update the weights and biases of the layer with the gradients and the learning rate
            layer.update(learning_rate)


In [750]:
neural_network = NeuralNetwork(50, 4, loss="categorical_crossentropy", optimizer="sgd")

# Add layers to the neural network
#neural_network.add_layer(64, activation="relu", weight_init="random")
neural_network.add_layer(32, activation="relu", weight_init="random")
neural_network.add_layer(4, activation="softmax", weight_init="random")

# Train the neural network with the whole data
neural_network.train(X.to_numpy(), y.to_numpy(), epochs=100, learning_rate=0.01)

inisiasi dweights (50, 32)
inisiasi dweights (32, 4)
Epoch 1, Loss: 0.42995621549529545
backprop  0
reverse init err (388, 4)
error size (388, 4)
a size (388, 4)
inputs.T <dot> error


AttributeError: 'Layer' object has no attribute 'inputs'

# Prediction

In [ ]:
x_test = X.to_numpy()[:10]
neural_network.evaluate(x_test)

AttributeError: 'NeuralNetwork' object has no attribute 'evaluate'